In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

from rubin_sim.data import get_data_dir
import sqlite3

from sat_utils import Constellation, starlink_constellation_v1

In [2]:
# load up some visits

limit = 900

dd = get_data_dir()
baseline_file = os.path.join(dd,'sim_baseline/baseline.db')

# Conenct to the sqlite database
con = sqlite3.connect(baseline_file)

# Load up the first year
df = pd.read_sql('select * from observations where night < 1 and sunAlt > -24 limit 15;', con)

con.close()

In [3]:
# data frame with a list of observations
df

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunAz,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id
0,0,310.024480,-60.812928,60218.001806,60218.023576,30.0,y,-297.249225,-297.249225,2,...,255.593220,186.644048,-2.870827,27.609463,11.956111,94.490314,102.958651,87.407902,169.454444,0
1,1,310.601871,-63.561425,60218.002254,60218.023576,30.0,y,-297.708278,-297.708278,2,...,255.500445,186.644453,-2.871001,27.615338,11.959438,95.029204,101.743959,87.404494,170.502875,0
2,2,311.292611,-66.317774,60218.002703,60218.023576,30.0,y,-297.909620,-297.909620,2,...,255.407493,186.644858,-2.871176,27.621208,11.962765,95.563446,100.497860,87.401088,171.406738,0
3,3,312.140731,-69.082666,60218.003152,60218.023576,30.0,y,-297.838337,-297.838337,2,...,255.314364,186.645264,-2.871350,27.627073,11.966093,96.092842,99.221261,87.397685,172.197791,0
4,4,304.170163,-73.375442,60218.003623,60218.023576,30.0,y,-309.290623,-309.290623,2,...,255.216260,186.645690,-2.871533,27.633232,11.969593,99.605370,94.821142,87.394110,177.239803,0
5,5,304.269558,-70.565222,60218.004072,60218.023576,30.0,y,-308.771894,-308.771894,2,...,255.122756,186.646096,-2.871707,27.639086,11.972923,99.093478,96.132959,87.390713,176.807505,0
6,6,310.024480,-60.812928,60218.004959,60218.023576,30.0,y,-297.249225,-297.249225,2,...,254.937589,186.646897,-2.872051,27.650630,11.979500,94.530102,102.956350,87.384011,169.721145,0
7,7,310.601871,-63.561425,60218.005408,60218.023576,30.0,y,-297.708278,-297.708278,2,...,254.843658,186.647302,-2.872225,27.656461,11.982828,95.067781,101.741752,87.380625,170.615190,0
8,8,311.292611,-66.317774,60218.005856,60218.023576,30.0,y,-297.909620,-297.909620,2,...,254.749551,186.647708,-2.872399,27.662286,11.986156,95.600722,100.495751,87.377242,171.382855,0
9,9,312.140731,-69.082666,60218.006305,60218.023576,30.0,y,-297.838337,-297.838337,2,...,254.655266,186.648113,-2.872573,27.668106,11.989485,96.128729,99.219254,87.373862,172.052509,0


In [4]:
# Make a satellite constellation
tles = starlink_constellation_v1()
constellation = Constellation(tles)

In [ ]:
# Check the first method
lengths = []
nstreaks = []
n_rows = len(df)
t1 = time.time()
for index, row in df.iterrows():
    length, streak = constellation.check_pointing(row['altitude'], row['azimuth'],
                                    row['observationStartMJD'], row['visitTime'])
    lengths.append(length)
    nstreaks.append(streak)
    # A simple progress bar
    progress = index/float(n_rows)*100
    text = "\rprogress = %.3f%%" % progress
    sys.stdout.write(text) 
    sys.stdout.flush() 
t2 = time.time()
print('runtime = %.2f min' % ((t2-t1)/60.) )

progress = 60.000%

In [ ]:
# check the new method
t1 = time.time()

fast_lengths, fast_nstreaks = constellation.check_pointings(df['fieldRA'].values, df['fieldDec'].values,
                                                           df['observationStartMJD'].values,
                                                           df['visitTime'].values)
t2 = time.time()
print('runtime = %.2f min' % ((t2-t1)/60.) )


In [ ]:
np.array(lengths)

In [ ]:
fast_lengths

In [ ]:
(fast_lengths-np.array(lengths))/fast_lengths

In [ ]:
fast_lengths, fast_nstreaks

In [ ]:
np.where(fast_nstreaks != nstreaks)

In [ ]:
# So, why is the fast method finding more streaks than the slow method?

In [ ]:
i=60


In [ ]:
fast_lengths, fast_nstreaks = constellation.check_pointings(df['fieldRA'].values[i], df['fieldDec'].values[i],
                                                           df['observationStartMJD'].values[i],
                                                           df['visitTime'].values[i])


In [ ]:
length, streak = constellation.check_pointing(df['altitude'].values[i], df['azimuth'].values[i],
                                    df['observationStartMJD'].values[i], df['visitTime'].values[i])

In [ ]:
streak, fast_nstreaks

In [ ]:
df['fieldRA'].values[i], df['fieldDec'].values[i]

In [ ]:
df['altitude'].values[i], df['azimuth'].values[i]

In [ ]:
ras1, decs1, alts1, illums1 = constellation.paths_array(df['observationStartMJD'].values[i])

In [ ]:
plt.plot(np.degrees(ras1), np.degrees(decs1), 'ko')
plt.plot(df['fieldRA'].values[i], df['fieldDec'].values[i], 'ro')

In [ ]:
constellation.update_mjd(df['observationStartMJD'].values[i])
plt.plot(np.degrees(constellation.azimuth_rad), np.degrees(constellation.altitudes_rad),'ko')
plt.plot(df['azimuth'].values[i], df['altitude'].values[i],  'ro')

In [ ]:
constellation.update_mjd(df['observationStartMJD'].values[i]+34/3600/24)
plt.plot(np.degrees(constellation.azimuth_rad), np.degrees(constellation.altitudes_rad),'ko')
plt.plot(df['azimuth'].values[i], df['altitude'].values[i],  'ro')